In [59]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from time import sleep

In [81]:
base_url = 'https://www.pbs.org/newshour/politics/page/'

topics = ["politics", "economy", "health", 'science', 'nation', 'education'] # we should add world

start_month = 'jul'
continue_processing = True
first_time_processing_month = True

current_date = ""
 
# want to get data from /1 to /33
# about 1 month of news articles 
# each page has 5-10 articles
# can adjust the range to get different date range

# def get_month_range():
#     month_range = []
#     for i in range(1, 34):
#         month_range.append(i)
#     return month_range


        
    

In [80]:
def strip_topic_from_string(string):
    global topics
    
    for topic in topics:
        string = string.lower().replace(topic.lower(), "")

    return string
    

In [61]:
#test getting url html for one page

sample = "https://www.pbs.org/newshour/politics/page/2"

response = requests.get(sample)

if response.status_code == 200:

    print('sucess')
else:
    print('Failed to retrieve the webpage. Status code:', response.status_code)

sucess


In [62]:

# get html for a page url

def get_html(url):
    response = requests.get(url)
    #print(response)
    if response.status_code == 200:
        return response.text
    else:
        return None

html = get_html('https://www.pbs.org/newshour/politics/page/1')
#print(html)


In [79]:
# parse articles and get title, link and date from each article on page
def parse_articles(html,topic):
    global first_time_processing_month
    global continue_processing
    global current_date
    global start_month
    soup = BeautifulSoup(html, 'html.parser')
    articles = soup.find('div', class_="archive__cards")
    articles = articles.find_all('article')
    

    if not articles:
        return None
    
    parsed_articles = []
    for article in articles:
        title_tag = article.find('a', class_=["card-xl__title", "card-lg__title", "card-horiz__title"])

        if not title_tag:
            continue

        title = title_tag.get_text(strip=True)
        link = title_tag['href']
        date_tag = article.find('p', class_=["card-horiz__meta", "card-xl__meta", "card-lg__meta"])
        date = date_tag.get_text(strip=True)
        date = strip_topic_from_string(date)

        if date.startswith(start_month) and not first_time_processing_month:
            continue_processing = False
            break

        if not date.startswith(start_month):
            first_time_processing_month = False
            
        current_date = date
        
        parsed_articles.append({
            'title': title,
            'link': link,
            'date': date
        })
        
    return parsed_articles
    
# parsed_articles = parse_articles(html)
# print(len(parsed_articles))

In [64]:

# from link url get article content 

def get_article_content(story_link):
    html_content = get_html(story_link)
    if not html_content:
        return None

    soup = BeautifulSoup(html_content, 'html.parser')
    #print(soup)
    storytext = soup.find("div", class_="body-text")
    if not storytext:
        return None
    
    content =[]
    paragraphs = storytext.find_all('p')
    for p in paragraphs:
        content.append(p.get_text(strip=True))

    return content

# article_content = get_article_content(parsed_articles[0]['link'])     
# print(article_content)

In [76]:
# get data from page range and return article information 

def get_articles_for_page(page, topic):
    url = f"https://www.pbs.org/newshour/{topic}/page/{page}"
    print(f"\n Processing page url: {url}")
    html = get_html(url)
    if not html:
        return None
    article_links = parse_articles(html, topic)



    articles_for_page = []
    for article_link in article_links:
        article_content = get_article_content(article_link['link'])
        article_link['content'] = article_content
        sleep(1) 
        articles_for_page.append(article_link)

    return articles_for_page

# articles = get_articles_for_page("2")
# print(articles)
# print(len(articles))
    

In [82]:
PBS_articles = pd.DataFrame()

global topics

global first_time_processing_month
global continue_processing
global current_date
global start_month

for topic in topics:
    page = 1
    continue_processing = True
    first_time_processing_month = True
    start_month = "jul"
    current_date = ""

    print(f"\nProcessing {topic} articles...")
    while continue_processing:
        articles = get_articles_for_page(page, topic)
        print(f"{len(articles)} articles added for topic: {topic} for date: {current_date}")
        PBS_articles = pd.concat([PBS_articles, pd.DataFrame(articles)])
        page += 1

    # save dataframe to pickle file
    PBS_articles.to_pickle(f"PBS_{topic}_articles.pkl")
    PBS_articles = pd.DataFrame()

    


Processing politics articles...

 Processing page url: https://www.pbs.org/newshour/politics/page/1
10 articles added for topic: politics for date: jul 06

 Processing page url: https://www.pbs.org/newshour/politics/page/2
10 articles added for topic: politics for date: jul 03

 Processing page url: https://www.pbs.org/newshour/politics/page/3
10 articles added for topic: politics for date: jul 03

 Processing page url: https://www.pbs.org/newshour/politics/page/4
10 articles added for topic: politics for date: jul 02

 Processing page url: https://www.pbs.org/newshour/politics/page/5
10 articles added for topic: politics for date: jul 01

 Processing page url: https://www.pbs.org/newshour/politics/page/6
10 articles added for topic: politics for date: jul 01

 Processing page url: https://www.pbs.org/newshour/politics/page/7
10 articles added for topic: politics for date: jun 30

 Processing page url: https://www.pbs.org/newshour/politics/page/8
10 articles added for topic: politics 